# Fine-mapping with SuSiE RSS model

This notebook take a list of LD file and a list of sumstat file and do salmon QC and susie RSS for each overlap LD block.

## Input

1. A tab delimated table describing the path where LD per region stored, can be generated using the ld_per_region_plink step of the genotype processing module.

```
#id     dir
chr17_60570445_65149278 /mnt/vast/hpc/csg/molecular_phenotype_calling/LD/output_npz_2/1300_hg38_EUR_LD_blocks_npz_files/ROSMAP_NIA_WGS.leftnorm.filtered.filtered.chr17_60570445_65149278.flt16.npz
```

2. A tab delimated table describing  path where summary stat per chromosome stored, can be generated using the yml_generator module before the qced sumstat are generated.
```
hs3163@csglogin:/mnt/vast/hpc/csg/xqtl_workflow_testing/susie_rss$ cat /mnt/vast/hpc/csg/xqtl_workflow_testing/ADGWAS/data_intergration/ADGWAS2022/qced_sumstat_list.txt
#chr    ADGWAS_Bellenguez_2022
1       /mnt/vast/hpc/csg/xqtl_workflow_testing/ADGWAS/data_intergration/ADGWAS2022/ADGWAS_Bellenguez_2022.1/ADGWAS2022.chr1.sumstat.tsv
2       /mnt/vast/hpc/csg/xqtl_workflow_testing/ADGWAS/data_intergration/ADGWAS2022/ADGWAS_Bellenguez_2022.2/ADGWAS2022.chr2.sumstat.tsv
3       /mnt/vast/hpc/csg/xqtl_workflow_testing/ADGWAS/data_intergration/ADGWAS2022/ADGWAS_Bellenguez_2022.3/ADGWAS2022.chr3.sumstat.tsv
4       /mnt/vast/hpc/csg/xqtl_workflow_testing/ADGWAS/data_intergration/ADGWAS2022/ADGWAS_Bellenguez_2022.4/ADGWAS2022.chr4.sumstat.tsv
5       /mnt/vast/hpc/csg/xqtl_workflow_testing/ADGWAS/data_intergration/ADGWAS2022/ADGWAS_Bellenguez_2022.5/ADGWAS2022.chr5.sumstat.tsv


```

## Output

1. A RDS file containing the output susie object, the name of all variants that went through the analysis, the z score , and the LD used for the analysis.
2. A sumstat file with additional column containing the slalom results.

## MWE

In [9]:
sos run pipeline/SuSiE_RSS.ipynb SuSiE_RSS \
    --LD_list test.ld.list \
    --sumstat_list /mnt/vast/hpc/csg/xqtl_workflow_testing/ADGWAS/data_intergration/ADGWAS2022/qced_sumstat_list.txt \
    --container containers/stephenslab.sif --impute --cwd output_impute_2 &

SyntaxError: invalid syntax (<ipython-input-9-158593d50ce7>, line 1)

In [ ]:
[global]
parameter: container = ""
parameter: entrypoint={('micromamba run -a "" -n' + ' ' + container.split('/')[-1][:-4]) if container.endswith('.sif') else f''}
# For cluster jobs, number commands to run per job
parameter: job_size = 1
# Wall clock time expected
parameter: walltime = "5h"
# Memory expected
parameter: mem = "16G"
# Number of threads
parameter: numThreads = 3
parameter: cwd = path("output")
# getting the overlapped input
parameter: LD_list = path
parameter: sumstat_list = path
import pandas as pd
LD_list = pd.read_csv(LD_list,sep="\t")
sumstat_list = pd.read_csv(sumstat_list,sep="\t")
LD_list["#chr"] = [x[0].replace("chr", "") for x in  LD_list["#id"].str.split("_") ]
sumstat_list["#chr"] = [str(x).replace("chr", "") for x in  sumstat_list["#chr"] ]
input_inv = LD_list.merge(sumstat_list)
input_list = input_inv.iloc[:,[1,3]].values.tolist()
parameter: lead_idx_choice = "pvalue"
parameter: abf_prior_variance = 0.4
parameter: nlog10p_dentist_s_threshold = 4
parameter: r2_threshold = 0.6
parameter: n = 0
parameter: max_iter = 1000
parameter: impute = True # Whether to impute the sumstat for all the snp in LD but not in sumstat.

In [ ]:
[SuSiE_RSS_1]
parameter: L = 10
parameter: max_L = 1000
input: input_list, group_by = 2
name = f'{_input[0]:b}'.split(".")[-3]
output: f'{cwd:a}/{_input[1]:bn}.{name}.unisusie_rss.fit.rds',
        f'{cwd:a}/{_input[1]:bn}.{name}.unisusie_rss.ss_qced.tsv'

task: trunk_workers = 1, trunk_size = job_size, walltime = walltime, mem = mem, cores = numThreads, tags = f'{step_name}_{_output[0]:bn}'
python: expand = '${ }', stdout = f"{_output[0]:nn}.stdout", stderr = f"{_output[0]:nn}.stderr", container = container, entrypoint = entrypoint
    import pandas as pd
    import numpy as np
    import rpy2
    from backports import zoneinfo
    import rpy2.robjects.numpy2ri as numpy2ri
    from rpy2.robjects import pandas2ri
    from rpy2.robjects.packages import SignatureTranslatedAnonymousPackage
    import rpy2.robjects as ro
    numpy2ri.activate()
    pandas2ri.activate()
    def load_npz_ld(path):
        np_ld_loaded = np.load(path,allow_pickle=True)
        # sort by start position
        snp_id = [x.replace(":","_") for x in np_ld_loaded.get("arr_1")]
        np_ld_loaded = np_ld_loaded.get("arr_0")
        new = np_ld_loaded + np_ld_loaded.T
        np.fill_diagonal(new, np.diag(new)/2)
        return new,snp_id

    def get_bcor_meta(bcor_obj):
        df_ld_snps = bcor_obj.getMeta()
        df_ld_snps.rename(columns={'rsid':'SNP', 'position':'BP', 'chromosome':'CHR', 'allele1':'A1', 'allele2':'A2'}, inplace=True, errors='raise')
        ###df_ld_snps['CHR'] = df_ld_snps['CHR'].astype(np.int64)
        df_ld_snps['BP'] = df_ld_snps['BP'].astype(np.int64)
        return df_ld_snps

    def load_ld_bcor(ld_prefix):
        bcor_file = ld_prefix+'.bcor'
        import os
        import time
        from ldstore.bcor import bcor
        if not os.path.exists(bcor_file):
            raise IOError('%s not found'%(bcor_file))
        t0 = time.time()
        bcor_obj = bcor(bcor_file)
        df_ld_snps = get_bcor_meta(bcor_obj)
        ld_arr = bcor_obj.readCorr([])
        assert np.all(~np.isnan(ld_arr))
        return ld_arr, df_ld_snps

    def abf(z, se, W=0.04):
        from scipy import special 
        V = se ** 2
        r = W / (W + V)
        lbf = 0.5 * (np.log(1 - r) + (r * z ** 2))
        denom = special.logsumexp(lbf)
        prob = np.exp(lbf - denom)
        return lbf, prob
    
    def get_cs(variant, prob, coverage=0.95):
        ordering = np.argsort(prob)[::-1]
        idx = np.where(np.cumsum(prob[ordering]) > coverage)[0][0]
        cs = variant[ordering][: (idx + 1)]
        return cs
    def slalom(df,LD,abf_prior_variance = 0.4 ,nlog10p_dentist_s_threshold = 4, r2_threshold = 0.6  ):
        from scipy import stats
        lbf, prob = abf(df.z, df.se if "se" in df.columns else 1, W=abf_prior_variance)
        cs = get_cs(df.variant, prob, coverage=0.95)
        cs_99 = get_cs(df.variant, prob, coverage=0.99)
        df["lbf"] = lbf
        df["prob"] = prob
        df["cs"] = df.variant.isin(cs)
        df["cs_99"] = df.variant.isin(cs_99)
        
        if "${lead_idx_choice}" == "pvalue":
            lead_idx_snp = df.pvalue.idxmin()
        else:
            lead_idx_snp = df.prob.idxmax()
            
        lead_variant = df.variant[lead_idx_snp]
        df["lead_variant"] = False
        df["lead_variant"].iloc[lead_idx_snp] = True
        # annotate LD     
        ## This is to identify the R for each snp vs the lead snp
        df["r"] = [LD[np.where(np.in1d(df.variant,lead_variant))][:,np.where(np.in1d(df.variant,x))][0][0][0] for x in df.variant]
        lead_z = df.z.iloc[lead_idx_snp]
        df["t_dentist_s"] = (df.z - df.r * lead_z) ** 2 / (1 - df.r ** 2)
        df["t_dentist_s"] = np.where(df["t_dentist_s"] < 0, np.inf, df["t_dentist_s"])
        df["t_dentist_s"].iloc[lead_idx_snp] = np.nan
        df["nlog10p_dentist_s"] = stats.chi2.logsf(df["t_dentist_s"], df=1) / -np.log(10)
        df["r2"] = df.r ** 2
        df["outliers"] = (df.r2 > r2_threshold) & (df.nlog10p_dentist_s > nlog10p_dentist_s_threshold)
        df_output = df
        n_r2 = np.sum(df.r2 > r2_threshold)
        n_dentist_s_outlier = np.sum(
            (df.r2 > r2_threshold) & (df.nlog10p_dentist_s > nlog10p_dentist_s_threshold)
        )
        max_pip_idx = df.prob.idxmax()
        df_summary = pd.DataFrame(
            {
                "lead_pip_variant": [df.variant.iloc[max_pip_idx]],
                "n_total": [len(df.index)],
                "n_r2": [n_r2],
                "n_dentist_s_outlier": [n_dentist_s_outlier],
                "fraction": [n_dentist_s_outlier / n_r2 if n_r2 > 0 else 0],
                "max_pip": [np.max(df.prob)]
            }
            )
        return df, df_summary
    
    ## Load LD
    if "${_input[0]}".endswith("npz"):
        LD,snp_id = load_npz_ld(${_input[0]:r}) 
    if "${_input[0]}".endswith("bcor"):
        LD,snp_id = load_ld_bcor(${_input[0]:nr}) 
        
    sumstat = pd.read_csv(${_input[1]:r}, sep="\t")
    # check if z column already exists. If not then create it
    if "z" not in sumstat.columns:
        sumstat["z"] = sumstat.beta / sumstat.se
    
    ## Remove nan in LD
    where = np.where(np.isnan(LD))
    na_snp = np.array(snp_id)[[x for x in where[0]] + [x for x in where[1]]]
    ### Remove NA from LD
    LD = LD[np.ix_(~np.in1d(snp_id, na_snp),~np.in1d(snp_id, na_snp))]
    ### Remove NA from snp_list
    snp_id = np.array(snp_id)[~np.in1d(snp_id, na_snp)]
    
    ## Get only intersect snp
    intersct = np.intersect1d(sumstat.variant.to_numpy(),snp_id)
    sumstat = sumstat.query("variant in @intersct").reset_index()
    indice = np.where(np.in1d(snp_id, intersct))
    ## slalom
    ss_qc,ss_qc_sum = slalom(sumstat,LD[np.ix_(indice[0].tolist(), indice[0].tolist())],${abf_prior_variance},${nlog10p_dentist_s_threshold},${r2_threshold})
    print(ss_qc_sum)
    ss_qc.to_csv(${_output[1]:r},"\t",index = 0 )
    ## Filter out outlier
    ### Get outliers
    outliers = ss_qc[ss_qc.outliers].variant
    ### Remove outliers from LD
    LD = LD[np.ix_(~np.in1d(snp_id, outliers),~np.in1d(snp_id, outliers))]
    ### Remove outliers from snp_list
    snp_id = np.array(snp_id)[~np.in1d(snp_id, outliers)]
    ss_qc = ss_qc[~ss_qc.outliers]

    if ${impute}:
        cur_LD = LD
        cur_miss = ~np.in1d(snp_id, ss_qc.variant)
        print(f'{np.mean(cur_miss)} of snps in the LD panel does not have sumstat. Their sumstat are imputed')
        ## Sumstat imputation based on fusion code
        cur_wgt = cur_LD[np.ix_(cur_miss, np.logical_not(cur_miss))].dot(np.linalg.inv(cur_LD[np.logical_not(cur_miss), np.logical_not(cur_miss)] + 0.1 * np.eye(np.sum(np.logical_not(cur_miss)))))
        cur_impz  = cur_wgt @ ss_qc.z
        cur_r2pred = np.diag(cur_wgt.dot(cur_LD[np.ix_(np.logical_not(cur_miss),np.logical_not(cur_miss))]).dot(cur_wgt.T))
        imputed_z = cur_impz/np.sqrt(cur_r2pred)
        working_ss = pd.concat([ss_qc[["variant","z"]], pd.DataFrame({"z" : imputed_z, "variant": np.array(snp_id)[cur_miss]})]).set_index("variant").reindex(snp_id).reset_index()
        import scipy.stats as st
        working_ss["beta"] = working_ss.z
        working_ss["se"] = 1
        working_ss["pvalue"] = (1 - st.norm.cdf(working_ss.z))*2
        
        
        working_ss_qc, working_ss_qc_sum = slalom(working_ss,cur_LD,0.4,4,0.6)
        print(working_ss_qc_sum)
        working_ss_qc.to_csv('${_output[1]:nn}.imputed_ss_qced.tsv',"\t",index = 0 )
        imputed_outliers = working_ss_qc[working_ss_qc.outliers].variant
        ### Remove outliers from LD
        cur_LD = LD[np.ix_(~np.in1d(snp_id, imputed_outliers),~np.in1d(snp_id, imputed_outliers))]
        ### Remove outliers from snp_list
        snp_id = np.array(snp_id)[~np.in1d(snp_id, imputed_outliers)]
        working_ss_qc = working_ss_qc[~working_ss_qc.outliers]
        ss_qc = working_ss_qc
        LD = cur_LD 
    ### For non-imputed LD, retained only the overlap snps. For imputed LD, this should do nothing as snp_id, ss_qc.variant .
    indice = np.where(np.in1d(snp_id, ss_qc.variant))
    LD = LD[np.ix_(indice[0].tolist(), indice[0].tolist())]
    ## SuSiERSS
    string="""
    susie_rss_analysis=function(ss_df, R, n, var_y, z_ld_weight = 0, estimate_residual_variance = FALSE, 
    prior_variance = 50, check_prior = TRUE, output_path,L = ${L} , max_iter  = ${max_iter} ){
    condz = susieR::kriging_rss(as.matrix(ss_df$z),R ,n = ${n})
    ggplot2::ggsave(plot = condz$plot,filename = "${_output[0]}.kriging_rss.pdf", device = "pdf")
    res = susieR:::susie_rss(as.matrix(ss_df$z),as.matrix(R), n = ${n},var_y, z_ld_weight = 0, estimate_residual_variance = FALSE, 
    prior_variance = 50, check_prior = TRUE, max_iter = ${max_iter},verbose = TRUE )
    res$variants = ss_df$variant
    res$z = ss_df$z
    res$corr = susieR:::get_cs_correlation(res, X = NULL, Xcorr = R, max = FALSE)
    res$conditional_dist = condz$conditional_dist 
    rownames(res$corr) <- names(res$cs)
    colnames(res$corr) <- names(res$cs)
    if (length(res$sets$cs) > 1) {
        index_combos = expand.grid(1:length(res$sets$cs),1:length(res$sets$cs))
        in_common = apply(index_combos, 1, function(x) intersect(res$sets$cs[[x[1]]], res$sets$cs[[x[2]]]))
        counts = unlist(lapply(in_common, length))
        ovlp_mat = matrix(counts, ncol = length(res$sets$cs), byrow = T)
        ovlp_mat[lower.tri(ovlp_mat)] = NA
        rownames(ovlp_mat) = names(res$sets$cs)
        colnames(ovlp_mat) = names(res$sets$cs)
        print(ovlp_mat)
        res$sets[["ovlp_mat"]] = ovlp_mat
    }
    saveRDS(res,output_path)
    return(res)
    }
    """
    susie_analysis = SignatureTranslatedAnonymousPackage(string, "susie_analysis")
    analysis_result = susie_analysis.susie_rss_analysis(ss_df = ss_qc,R = LD,output_path=${_output[0]:r} ${f', n ={n}' if n > 2 else ""})

In [ ]:
[SuSiE_RSS_2]
output: pip_plot = f"{cwd}/{_input:bn}.png"
task: trunk_workers = 1, trunk_size = job_size, walltime = '12h', mem = '20G', cores = numThreads, tags = f'{step_name}_{_output:bn}'
R: container=container, expand = "${ }", stderr = f'{_output[0]:n}.stderr', stdout = f'{_output[0]:n}.stdout', entrypoint = entrypoint
    res = readRDS(${_input:r})
    png(${_output[0]:r}, width = 14, height=6, unit='in', res=300)
    par(mfrow=c(1,2))
    susieR::susie_plot(res, y= "PIP", pos=list(attr='pos',start=res$pos[1],end=res$pos[length(res$pos)]), add_legend=T, xlab="position")
    susieR::susie_plot(res, y= "z", pos=list(attr='pos',start=res$pos[1],end=res$pos[length(res$pos)]), add_legend=T, xlab="position", ylab="-log10(p)")
    dev.off()

In [ ]:
[SuSiE_RSS_3]
sep = "" #'\n\n---\n'
input: group_by = 'all'
output: analysis_summary = f'{cwd}/{sumstats_path:bnn}.analysis_summary.md', variants_csv = f'{cwd}/{sumstats_path:bnn}.variants.csv'
python: container=container, expand = "${ }", entrypoint = entrypoint

    theme = '''---
    theme: base-theme
    style: |
     p {
       font-size: 24px;
       height: 900px;
       margin-top:1cm;
      }
      img {
        height: 70%;
        display: block;
        margin-left: auto;
        margin-right: auto;
      }
      body {
       margin-top: auto;
       margin-bottom: auto;
       font-family: verdana;
      }
    ---    
    '''
    import numpy as np
    import pandas as pd
    
    # will load the rds file outputted in a previous step
    def load_rds(filename, types=None):
        import os
        import pandas as pd, numpy as np
        import rpy2.robjects as RO
        import rpy2.robjects.vectors as RV
        import rpy2.rinterface as RI
        from rpy2.robjects import numpy2ri
        numpy2ri.activate()
        from rpy2.robjects import pandas2ri
        pandas2ri.activate()
        def load(data, types, rpy2_version=3):
            if types is not None and not isinstance(data, types):
                return np.array([])
            # FIXME: I'm not sure if I should keep two versions here
            # rpy2_version 2.9.X is more tedious but it handles BoolVector better
            # rpy2 version 3.0.1 converts bool to integer directly without dealing with
            # NA properly. It gives something like (0,1,-234235).
            # Possibly the best thing to do is to open an issue for it to the developers.
            if rpy2_version == 2:
                # below works for rpy2 version 2.9.X
                if isinstance(data, RI.RNULLType):
                    res = None
                elif isinstance(data, RV.BoolVector):
                    data = RO.r['as.integer'](data)
                    res = np.array(data, dtype=int)
                    # Handle c(NA, NA) situation
                    if np.sum(np.logical_and(res != 0, res != 1)):
                        res = res.astype(float)
                        res[res < 0] = np.nan
                        res[res > 1] = np.nan
                elif isinstance(data, RV.FactorVector):
                    data = RO.r['as.character'](data)
                    res = np.array(data, dtype=str)
                elif isinstance(data, RV.IntVector):
                    res = np.array(data, dtype=int)
                elif isinstance(data, RV.FloatVector):
                    res = np.array(data, dtype=float)
                elif isinstance(data, RV.StrVector):
                    res = np.array(data, dtype=str)
                elif isinstance(data, RV.DataFrame):
                    res = pd.DataFrame(data)
                elif isinstance(data, RV.Matrix):
                    res = np.matrix(data)
                elif isinstance(data, RV.Array):
                    res = np.array(data)
                else:
                    # I do not know what to do for this
                    # But I do not want to throw an error either
                    res = str(data)
            else:
                if isinstance(data, RI.NULLType):
                    res = None
                else:
                    res = data
            if isinstance(res, np.ndarray) and res.shape == (1, ):
                res = res[0]
            return res
        def load_dict(res, data, types):
            '''load data to res'''
            names = data.names if not isinstance(data.names, RI.NULLType) else [
                i + 1 for i in range(len(data))
            ]
            for name, value in zip(names, list(data)):
                if isinstance(value, RV.ListVector):
                    res[name] = {}
                    res[name] = load_dict(res[name], value, types)
                else:
                    res[name] = load(value, types)
            return res
        #
        if not os.path.isfile(filename):
            raise IOError('Cannot find file ``{}``!'.format(filename))
        rds = RO.r['readRDS'](filename)
        if isinstance(rds, RV.ListVector):
            res = load_dict({}, rds, types)
        else:
            res = load(rds, types)
        return res
    
    def f7(seq):
        seen = set()
        seen_add = seen.add
        return [x for x in seq if not (x in seen or seen_add(x))]



    text = ""
    sep = '\n\n---\n'
    
    inp = "${_input:r}".split(" ")
    for i, each in enumerate(inp):
        inp[i] = ".".join(each.split(".")[:-1])

    r = f7("${_input:bn}".split(" "))
    
    num_csets = []
    region_info = []
    
    # this will be a 2d array that stores information about each variant of interest in the phenotype
    # this includes all the variants in a cs and all the variants past the cutoff
    variant_info = []

    for reg_i, each in enumerate(f7(inp)):
    
        rid = r[reg_i].split('.')[0]
        
        text_temp = ""
        text_temp += "#\n\n SuSiE RSS {region} \n".format(region=r[reg_i])
        text_temp += "![]({region}.png){sep} \n \n".format(region=r[reg_i], sep=sep)

        rd = load_rds(each[1:]+".rds")
        
        # find the number of cs in the current region
        if rd["sets"]["cs"] == None:
            num_csets.append(0)
        else:
            num_csets.append(len(rd["sets"]["cs"]))
        print(num_csets)
        
        # this will store the indicies of all variants that cross the threshold
        ind_p = []

        pval = ${pip_cutoff}

        for i, each in enumerate(rd["pip"]):
            if each >= pval:
                ind_p.append(i)
        sumvars = 0
        
        # if we have at least one cs in the current region
        if num_csets[reg_i] > 0:
            tbl_header = "| chr number | pos at highest pip | ref | alt | region id | cs | highest pip |  \n"
            tbl_header += "| --- | --- | --- | --- | --- | --- | --- |  \n"

            table = ""
            
            sumpips = 0
            
            for cset in rd["sets"]["cs"].keys():
                print(cset)
                
                # if we have many variants in the cs
                if isinstance(rd["sets"]["cs"][cset], np.ndarray):
                    highestpip = 0
                    poswhighestpip = -1
                    for i in rd["sets"]["cs"][cset]:
                        i = i.item() - 1
                        
                        # we make sure that ind_p only stores the variants that aren't in any cs
                        if i in ind_p: ind_p.remove(i) 
                        
                        # append variant info
                        variant_info.append( [rd["chr"][i], rd["pos"][i], rd["ref"][i], rd["alt"][i], rid, cset, rd["pip"][i]] )
                        
                        if rd["pip"][i] > highestpip:
                            highestpip = rd["pip"][i]
                            poswhighestpip = i
                            
                        sumpips += rd["pip"][i]
                        sumvars += 1
                        
                    if poswhighestpip > -1:
                        i = poswhighestpip
                        table += "| {chr} | {pos} | {ref} | {alt} | {rid} | {cs} | {pip:.2f} |  \n".format(chr=rd["chr"][i], pos=rd["pos"][i], ref=rd["ref"][i], alt=rd["alt"][i], rid=rid, cs=cset, pip=rd["pip"][i])
                
                else: # if we have only one variant in the cs
                    i =  rd["sets"]["cs"][cset]
                    i = i.item() - 1
                    
                    # we make sure that ind_p only stores the variants that aren't in any cs
                    if i in ind_p: ind_p.remove(i)
                    
                    # append variant info
                    variant_info.append( [rd["chr"][i], rd["pos"][i], rd["ref"][i], rd["alt"][i], rid, cset, rd["pip"][i]] )
                    
                    table += "| {chr} | {pos} | {ref} | {alt} | {rid} | {cs} | {pip:.2f} |  \n".format(chr=rd["chr"][i], pos=rd["pos"][i], ref=rd["ref"][i], alt=rd["alt"][i], rid=rid, cs=cset, pip=rd["pip"][i])
                    
                    sumpips += rd["pip"][i]
                    sumvars += 1
            

            text_temp += "- Total number of variants: {}\n".format(len(rd["pip"]))
            text_temp += "- Expected number of causal variants: {:.2f}\n".format(sumpips)
            text_temp += "- Number of variants with PIP > {} and not in any CS: {}\n\n".format(pval, len(ind_p))
            text_temp += tbl_header + table + sep
            
            if num_csets[reg_i] > 1:
                text_temp += "#### CORR: Correlation between CS | OLAP: Overlap between CS\n"
                
                cs = list(rd["sets"]["cs"].keys())

                corrheader = "|  |"
                corrbreak = "| --- |"

                for i in cs:
                    corrheader += " CORR {} |".format(i)
                    corrbreak += " --- |"
                    
                corrheader += "  |"
                corrbreak += " --- |"
                    
                for i in cs:
                    corrheader += " OLAP {} |".format(i)
                    corrbreak += " --- |"

                corrheader += "\n"
                corrbreak += "\n"

                body = ""

                for en, i in enumerate(cs):
                    body += "| {} |".format(i)
                    for j in rd["cscorr"][en]:
                        body += " {:.2f} |".format(j)
                    body += "  |"
                    for j in rd["sets"]["cs"]:
                        body += " {} |".format(len(np.intersect1d(rd["sets"]["cs"][i], rd["sets"]["cs"][j])))
                    body += "\n"
                
                text_temp += corrheader + corrbreak + body + sep
            
        region_info.append(text_temp)
            
    f = open(${_output["analysis_summary"]:r}, "w")
    
    cset_order = np.argsort(num_csets)
    cset_order = cset_order.tolist()
    cset_order.reverse()
    for c in cset_order:
        text += region_info[c]
    
    f.write(theme + text)
    
    f.close()
    
    for i in ind_p:
        # append variant info
        variant_info.append( [rd["chr"][i], rd["pos"][i], rd["ref"][i], rd["alt"][i], rid, "None", rd["pip"][i]] )
        
    df = pd.DataFrame(variant_info, columns=["chr", "pos", "ref", "alt", "rid", "cs", "pip"])
    df.to_csv(${_output["variants_csv"]:r}, sep = "\t", header = True, index = True)

In [ ]:
# Generate analysis report: HTML file, and optionally PPTX file
[SuSiE_RSS_4]
output: f"{_input['analysis_summary']:n}.html"
sh: container=container_marp, expand = "${ }", stderr = f'{_output:n}.stderr', stdout = f'{_output:n}.stdout', entrypoint = entrypoint
    node /opt/marp/.cli/marp-cli.js ${_input['analysis_summary']} -o ${_output:a} \
        --title '${region_file:bnn} fine mapping analysis' \
        --allow-local-files
    node /opt/marp/.cli/marp-cli.js ${_input['analysis_summary']} -o ${_output:an}.pptx \
        --title '${region_file:bnn} fine mapping analysis' \
        --allow-local-files